### Reqirements
- keras >= 2.2.0 or tensorflow >= 1.13
- segmenation-models==1.0.*
- albumentations==0.3.0


# Loading dataset

In [1]:
#!pip install tiffile --quiet

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import glob
from tiffile import imsave

import cv2
import tensorflow
import keras
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A

import segmentation_models as sm

import tensorflow as tf
tf.test.gpu_device_name()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

Segmentation Models: using `keras` framework.
Num GPUs Available:  1


In [2]:
DATA_DIR = '/data/'

# Debug for fileloading
FLAG_DEBUG_LOADING = 0

# Single z stack
FLAG_SINGLE_Z = 1
FLAG_3CLASS = 1

# Convert image to imtype 
imtype = "uint8"
# imtype = "float32"

class_weights=np.array([0 , 1 , .5])
class_labels = ['background' , 'nucleus' , 'cytoplasm']
CLASSES = class_labels


dir_tag = ''
tag = ''

x_train_dir = [os.path.join(DATA_DIR, 'Image_BF1_train') , os.path.join(DATA_DIR, 'Image_BF2_train') , os.path.join(DATA_DIR, 'Image_BF3_train')]
y_train_dir = os.path.join(DATA_DIR, 'Mask_3class_train' + dir_tag)

x_valid_dir = [os.path.join(DATA_DIR, 'Image_BF1_dev') , os.path.join(DATA_DIR, 'Image_BF2_dev') , os.path.join(DATA_DIR, 'Image_BF3_dev')]
y_valid_dir = os.path.join(DATA_DIR, 'Mask_3class_dev' + dir_tag)

x_test_dir = [os.path.join(DATA_DIR, 'Image_BF1_test') , os.path.join(DATA_DIR, 'Image_BF2_test') , os.path.join(DATA_DIR, 'Image_BF3_test')]
y_test_dir = os.path.join(DATA_DIR, 'Mask_3class_test' + dir_tag)

# Dataloader and utility functions 

In [3]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(50, 50))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, np.ceil(n / 1), i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = class_labels
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            n_images=None
    ):
        # Make sure files are present in all folders
        self.ids1 = [os.path.basename(x) for x in glob.glob(images_dir[1] + '/*.tif')]
        self.ids3 = [os.path.basename(x) for x in glob.glob(masks_dir + '/*.tif')]
        self.ids = list(set(self.ids1) & set(self.ids3))

        # # Number of training examples
        if len(self.ids) > n_images:
            self.ids = self.ids[0:n_images]

        self.images_fps0 = [os.path.join(images_dir[0], image_id) for image_id in self.ids]
        self.images_fps1 = [os.path.join(images_dir[1], image_id) for image_id in self.ids]
        self.images_fps2 = [os.path.join(images_dir[2], image_id) for image_id in self.ids]

        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        if FLAG_DEBUG_LOADING == 1:
            print(self.masks_fps[i])
            
        # read data and convert
        if FLAG_SINGLE_Z == 0:
            image0 = cv2.imread(self.images_fps0[i] , cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)
            image1 = cv2.imread(self.images_fps1[i] , cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)
            image2 = cv2.imread(self.images_fps2[i] , cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)
        else:
            image1 = cv2.imread(self.images_fps1[i] , cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)
            image0 = image1
            image2 = image1
        image = np.stack((image0 , image1 , image2) , axis = 2)

        if imtype == "uint8":
            image = cv2.convertScaleAbs(image , alpha = (255. / 65535.))
        if imtype == "float32":
            image = image.astype("float32")

        # Resize image
        # print(image)
        if FLAG_RESIZE == 1:
            image = cv2.resize(image , (imsize , imsize))

        if FLAG_DEBUG_LOADING == 1:
            print(self.masks_fps[i])
        mask = cv2.imread(self.masks_fps[i], cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)

        # Resize mask using nearest neighbor to avoid decimal points
        if FLAG_RESIZE == 1:
            mask = cv2.resize(mask , dsize = (imsize , imsize) , interpolation = cv2.INTER_NEAREST)
        
        # extract certain classes from mask (e.g. cars)
        # print(self.class_values)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary: not necessary since every pixel is labeled
        # if mask.shape[-1] != 1:
        #     background = 1 - mask.sum(axis=-1, keepdims=True)
        #     mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return batch
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)   
            
            
### Augmentations
# def round_clip_0_1(x, **kwargs):
#     return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),

        A.VerticalFlip(p=0.5),

        A.Transpose(p=0.5),

        # A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        A.PadIfNeeded(min_height=imsize, min_width=imsize, always_apply=True, border_mode=0),
        A.RandomCrop(height=imsize, width=imsize, always_apply=True),

    ]
    return A.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(imsize_test, imsize_test)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [4]:
### other stuff
# Random crops
FLAG_RESIZE = 0
LR = 0.0001
imsize_test = 2176
n_images_dev = 150
n_images_test = 150
n_images_train = 1500
imsize = 640
BATCH = 4

# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES))  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

# load best weights
from keras.models import load_model

model_dir = '/data/models'
model_filenames = os.listdir(model_dir)
# print(model_filenames)
for temp in range(len(model_filenames)): 
    print(str(temp) + ':' + model_filenames[temp])

0:models_vgg16_16_640_4_3750_2020-03-07 01:22:57.127240
1:models_vgg16_128_640_4_468_2020-03-07 05:55:33.992073
2:models_vgg16_512_640_4_117_2020-03-07 10:21:15.504758
3:models_vgg16_1024_640_4_58_2020-03-07 14:44:55.288484
4:models_vgg16_1500_640_4_40_2020-03-07 19:06:47.991860
5:models_efficientnetb4_16_320_8_3750_2020-03-08 00:24:44.388484
6:models_efficientnetb4_128_320_8_468_2020-03-08 04:53:37.271104
7:models_efficientnetb4_512_320_8_117_2020-03-08 09:16:00.047692
8:models_efficientnetb4_1024_320_8_58_2020-03-08 13:40:57.329230
9:models_efficientnetb4_1500_320_8_40_2020-03-08 18:06:30.098173
10:models_focal_vgg16_1500_640_4_40_2020-03-09 17:32:52.239663
11:models_dice_vgg16_1500_640_4_40_2020-03-09 18:48:40.628932
12:models_fullval_vgg16_1500_640_4_40_2020-03-09 21:16:35.334652
13:models_fullval_efficientnetb4_1500_320_8_40_2020-03-09 23:10:08.517890
14:models_4class_vgg16_1500_640_4_40_2020-03-10 22:02:04.105286


# Model Evaluation VGG16

In [8]:
model_loop = range(14)
backbone_loop = ['vgg16','vgg16','vgg16','vgg16','vgg16','efficientnetb4','efficientnetb4','efficientnetb4','efficientnetb4','efficientnetb4','vgg16','vgg16','vgg16','efficientnetb4']
loss_loop = ['hybrid','hybrid','hybrid','hybrid','hybrid','hybrid','hybrid','hybrid','hybrid','hybrid','focal','dice','hybrid','hybrid']

for mi, nums in enumerate(model_loop):
    idx = nums
    BACKBONE = backbone_loop[mi]
    print(model_filenames[idx])
    preprocess_input = sm.get_preprocessing(BACKBONE)

    test_dataset = Dataset(
        x_valid_dir, 
        y_valid_dir, 
        classes=CLASSES, 
        augmentation=get_validation_augmentation(),
        preprocessing=get_preprocessing(preprocess_input),
        n_images = n_images_dev
    )

    test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)

    #create model
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

    ### Load model
    # model = create_model()
    # model.summary()



    # model = keras.models.load_model(model_dir + '/' + model_filenames[idx])
    model.load_weights(model_dir + '/' + model_filenames[idx]+'/best_model_weights.h5')
    example_dir = model_dir + '/' + model_filenames[idx] + '/example/'
    if not os.path.exists(example_dir):
                os.makedirs(example_dir)
    
    ## predict images        
    showimage = 0
    saveimage = 0

    # n = 5
    # ids = np.random.choice(np.arange(len(test_dataset)), size=n)
    #ids = range(4)
    # ids = range(10,13)
    ids = []

    for i in ids:

        print(test_dataset.ids[i])

        image, gt_mask = test_dataset[i]
        image = np.expand_dims(image, axis=0)
        pr_mask = model.predict(image)

        if saveimage == 1:
            imsave(example_dir + test_dataset.ids[i][0:-4] + '_groundtruth.tif' , (gt_mask[8:2168 , 8:2168 , :] * 65535).astype('uint16'))
            #print('Ground truth mask saved.')
            imsave(example_dir + test_dataset.ids[i][0:-4] + '_predicted.tif' , (pr_mask[0 , 8:2168 , 8:2168 , :] * 65535).astype('uint16'))
            #print('Predicted mask saved.')

        if showimage == 1:   
            visualize(
            image=denormalize(image.squeeze()),
            ground_truth_mask=(gt_mask.squeeze()),
            #ground_truth_mask=(gt_mask[:,:,1:].squeeze()),
            # ground_truth_mask=(gt_mask[:,:,:].squeeze()),

            prediction_mask=(pr_mask.squeeze()),
            #prediction_mask=(pr_mask[:,:,:,1:].squeeze()),
            # prediction_mask=(pr_mask[:,:,:,:].squeeze()),

            # background_mask=pr_mask[..., 0].squeeze(),
            nucleus_mask=pr_mask[..., 1].squeeze(),
            # cytoplasm_mask=pr_mask[..., 2].squeeze(),
            # artifact_mask=pr_mask[..., 3].squeeze()
        )


    # run test set 
    LR = 0.0001
    optim = keras.optimizers.Adam(LR)

    # define loss
    dice_loss = sm.losses.DiceLoss(class_weights) 
    focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
    if loss_loop[mi] is 'hybrid':
        total_loss = dice_loss + (1 * focal_loss)
        print(loss_loop[mi] + ' loss')
    elif loss_loop[mi] is 'dice':
        total_loss = dice_loss
        print(loss_loop[mi] + ' loss')
    elif loss_loop[mi] is 'focal':
        total_loss = focal_loss
        print(loss_loop[mi] + ' loss')
    else:
        print('error in loss')


    # compile keras model with defined optimozer, loss and metrics
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
    model.compile(optim, total_loss, metrics)
    scores = model.evaluate_generator(test_dataloader , verbose = 1 , use_multiprocessing = False , workers = 8)
    print("Loss: {:.5}".format(scores[0]))
    for metric, value in zip(metrics, scores[1:]):
        print("mean {}: {:.5}".format(metric.__name__, value))

    # for nuclear
    class_weights_eval = np.array([0 ,3 , 0 ])
    metrics2 = [sm.metrics.IOUScore(threshold=0.5, class_weights = class_weights_eval), sm.metrics.FScore(threshold=0.5, class_weights = class_weights_eval)]
    model.compile(optim, total_loss, metrics2)
    scores2 = model.evaluate_generator(test_dataloader , verbose = 1 , use_multiprocessing = False , workers = 8)
    print("Loss: {:.5}".format(scores[0]))
    for metric, value in zip(metrics2, scores2[1:]):
        print("nuclear {}: {:.5}".format(metric.__name__, value))
    
    

models_vgg16_16_640_4_3750_2020-03-07 01:22:57.127240
hybrid loss
150/150 [==============================] - 63s 420ms/step
Loss: 0.20237
mean iou_score: 0.74797
mean f1-score: 0.83107
150/150 [==============================] - 63s 420ms/step
Loss: 0.20237
nuclear iou_score: 0.7471
nuclear f1-score: 0.85455
models_vgg16_128_640_4_468_2020-03-07 05:55:33.992073
hybrid loss
150/150 [==============================] - 63s 422ms/step
Loss: 0.1862
mean iou_score: 0.76683
mean f1-score: 0.84199
150/150 [==============================] - 63s 422ms/step
Loss: 0.1862
nuclear iou_score: 0.78066
nuclear f1-score: 0.8762
models_vgg16_512_640_4_117_2020-03-07 10:21:15.504758
hybrid loss
150/150 [==============================] - 65s 435ms/step
Loss: 0.18516
mean iou_score: 0.77483
mean f1-score: 0.84689
150/150 [==============================] - 64s 429ms/step
Loss: 0.18516
nuclear iou_score: 0.79436
nuclear f1-score: 0.88487
models_vgg16_1024_640_4_58_2020-03-07 14:44:55.288484
hybrid loss
150/150 